In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
%matplotlib inline

# PCA

### Derivation

* Find the covariance matrix - an empirical approximation.
* Decompose it into ...
* Find eigenvectors
$$
\begin{align}
 \mathbf{Q}_{\mathbf{XY}} = \frac{1}{n-1} \mathbf{M}_{\mathbf{X}}^T \mathbf{M}_{\mathbf{Y}} \\
\end{align}
$$


##### Resources
* https://en.wikipedia.org/wiki/Covariance_matrix
* http://www.cs.otago.ac.nz/cosc453/student_tutorials/principal_components.pdf
* http://mengnote.blogspot.co.nz/2013/05/an-intuitive-explanation-of-pca.html




### Singular value decomposition




In [3]:
### The mnist data set
from fuel.datasets import MNIST

def get_data(dtype,n):
    mnist = MNIST((dtype,))
    state = mnist.open()
    
    req = [i for i in range(n)] 
    im,_ = mnist.get_data(state=state, request=req)
    return im.reshape(im.shape[0],28*28)/255

train_ims = get_data('train',10000)
test_ims = get_data('test',1000)


pca = PCA(n_components=5,whiten=False)
pca.fit(train_ims)
#Check how well our model fits to training data vs test data vs noise
print(pca.score(train_ims))
print(pca.score(test_ims))
print(pca.score(np.random.random(test_ims.shape)))
#how can you have positive log likelihoods?

93.345071817
94.9298081682
-1556.38136467


In [4]:
test_score = pca.score_samples(test_ims)
noise_score = pca.score_samples(np.random.random(test_ims.shape))
print(np.min(test_score))
print(np.max(noise_score))

-359.553911194
-1331.7979641


In [5]:
pca.explained_variance_
#this seems like a lot of variance given we are in [0,1]

array([ 5.3807342 ,  3.80629998,  3.22355518,  2.85686354,  2.51950958])

In [6]:
kpca = KernelPCA(kernel='sigmoid')
kpca.fit(train_ims)



KeyboardInterrupt: 